In [ ]:
# 求整数的根号，保留整数部门
def sqrt(n):
    if n<=1:
        return n
    left=0
    right=n 
    while left<=right:
        mid=(left+right)//2
        if mid*mid==n:
            return mid
        elif mid*mid<n:
            left=mid+1
        import argparse
        import gym
        import numpy as np
        from itertools import count
        from collections import namedtuple
        
        import torch
        import torch.nn as nn
        import torch.nn.functional as F
        import torch.optim as optim
        from torch.distributions import Categorical
        
        # Cart Pole
        
        parser = argparse.ArgumentParser(description='PyTorch actor-critic example')
        parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                            help='discount factor (default: 0.99)')
        parser.add_argument('--seed', type=int, default=543, metavar='N',
                            help='random seed (default: 543)')
        parser.add_argument('--render', action='store_true',
                            help='render the environment')
        parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                            help='interval between training status logs (default: 10)')
        args = parser.parse_args()
        
        
        env = gym.make('CartPole-v0')
        env.seed(args.seed)
        torch.manual_seed(args.seed)
        
        
        SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
        
        
        class Policy(nn.Module):
            """
            implements both actor and critic in one model
            """
            def __init__(self):
                super(Policy, self).__init__()
                self.affine1 = nn.Linear(4, 128)
        
                # actor's layer
                self.action_head = nn.Linear(128, 2)
        
                # critic's layer
                self.value_head = nn.Linear(128, 1)
        
                # action & reward buffer
                self.saved_actions = []
                self.rewards = []
        
            def forward(self, x):
                """
                forward of both actor and critic
                """
                x = F.relu(self.affine1(x))
        
                # actor: choses action to take from state s_t
                # by returning probability of each action
                action_prob = F.softmax(self.action_head(x), dim=-1)
        
                # critic: evaluates being in the state s_t
                state_values = self.value_head(x)
        
                # return values for both actor and critic as a tupel of 2 values:
                # 1. a list with the probability of each action over the action space
                # 2. the value from state s_t
                return action_prob, state_values
        
        
        model = Policy()
        optimizer = optim.Adam(model.parameters(), lr=3e-2)
        eps = np.finfo(np.float32).eps.item()
        
        
        def select_action(state):
            state = torch.from_numpy(state).float()
            probs, state_value = model(state)
        
            # create a categorical distribution over the list of probabilities of actions
            m = Categorical(probs)
        
            # and sample an action using the distribution
            action = m.sample()
        
            # save to action buffer
            model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
        
            # the action to take (left or right)
            return action.item()
        
        
        def finish_episode():
            """
            Training code. Calcultes actor and critic loss and performs backprop.
            """
            R = 0
            saved_actions = model.saved_actions
            policy_losses = [] # list to save actor (policy) loss
            value_losses = [] # list to save critic (value) loss
            returns = [] # list to save the true values
        
            # calculate the true value using rewards returned from the environment
            for r in model.rewards[::-1]:
                # calculate the discounted value
                R = r + args.gamma * R
                returns.insert(0, R)
        
            returns = torch.tensor(returns)
            returns = (returns - returns.mean()) / (returns.std() + eps)
        
            for (log_prob, value), R in zip(saved_actions, returns):
                advantage = R - value.item()
        
                # calculate actor (policy) loss
                policy_losses.append(-log_prob * advantage)
        
                # calculate critic (value) loss using L1 smooth loss
                value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))
        
            # reset gradients
            optimizer.zero_grad()
        
            # sum up all the values of policy_losses and value_losses
            loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
        
            # perform backprop
            loss.backward()
            optimizer.step()
        
            # reset rewards and action buffer
            del model.rewards[:]
            del model.saved_actions[:]
        
        
        def main():
            running_reward = 10
        
            # run inifinitely many episodes
            for i_episode in count(1):
        
                # reset environment and episode reward
                state = env.reset()
                ep_reward = 0
        
                # for each episode, only run 9999 steps so that we don't
                # infinite loop while learning
                for t in range(1, 10000):
        
                    # select action from policy
                    action = select_action(state)
        
                    # take the action
                    state, reward, done, _ = env.step(action)
        
                    if args.render:
                        env.render()
        
                    model.rewards.append(reward)
                    ep_reward += reward
                    if done:
                        break
        
                # update cumulative reward
                running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        
                # perform backprop
                finish_episode()
        
                # log results
                if i_episode % args.log_interval == 0:
                    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                          i_episode, ep_reward, running_reward))
        
                # check if we have "solved" the cart pole problem
                if running_reward > env.spec.reward_threshold:
                    print("Solved! Running reward is now {} and "
                          "the last episode runs to {} time steps!".format(running_reward, t))
                    break
        
        
        if __name__ == '__main__':
            main()


In [3]:
def sqrt(n):
    if n<=1:
        return n 
    left=0
    right=n
    while left<=right:
        mid=(left+right)//2
        if mid*mid==n:
            return mid
        elif mid*mid<n:
            left=mid+1
        else:
            right=mid-1
    return right
sqrt(3)

1

In [10]:
# 合并k个升序列表
import heapq
def merge_k_sortnums(lsits):
    n=len(lsits)
    heap=[]
    res=[]
    for i,nums in enumerate(lsits):
        heapq.heappush(heap,(nums[0],i,0))
    while heap:
        num,index,start=heapq.heappop(heap)
        res.append(num)
        if start<len(lsits[index])-1:
            heapq.heappush(heap,(lsits[index][start+1],index,start+1))
    return res

merge_k_sortnums([[1, 4, 5], [1, 3, 4], [2, 6]])

    

[1, 1, 2, 3, 4, 4, 5, 6]

### 1. 排序方法
- 选择排序
- 冒泡排序
- 快速排序
- 归并排序
- 堆排序

In [14]:
# 选择排序
def choice_sort(nums):
    n=len(nums)
    for i in range(n-1,-1,-1):
        temp=i
        for j in range(i):
            if nums[j]>nums[temp]:
                temp=j 
        nums[temp],nums[i]=nums[i],nums[temp]
    return nums
nums=[3,1,28,7,6]
choice_sort(nums)

[1, 3, 6, 7, 28]

In [17]:
# 冒泡排序
def sort(nums):

    n=len(nums)

    for i in range(n-1,-1,-1):
        for j in range(i):
            if nums[j]>nums[j+1]:
                nums[j],nums[j+1]=nums[j+1],nums[j]
    return nums
nums=[3,1,28,7,6]
sort(nums)

[1, 3, 6, 7, 28]

In [19]:
# 快速排序
def quick_sort(nums):
    if len(nums)<=1:
        return nums
    privot=nums[0]
    
    left=[num for num in nums if num<privot]
    middle=[num for num in nums if num==privot]
    right=[num for num in nums if num>privot]
    return quick_sort(left)+middle+quick_sort(right)
nums=[3,1,28,7,6]
quick_sort(nums)

[1, 3, 6, 7, 28]

In [24]:
# 归并排序
def merge(left,rigght):
    res=[]
    while left and rigght:
        if left[0]<rigght[0]:
            res.append(left.pop(0))
        else:
            res.append(rigght.pop(0))
    if left:
        res=res+left
    if rigght:
        res=res+rigght
    return res
def split(nums):
    if len(nums)<=1:
        return nums
    n=len(nums)
    mid=n//2
    left=nums[:mid]
    right=nums[mid:]
    return merge(split(left),split(right))
nums=[3,1,28,7,6]
split(nums)
# nums1=[1,3,4]
# nums2=[1,2]
# merge(nums1,nums2)
    

[1, 3, 6, 7, 28]

In [3]:
# 华为笔试题2
from collections import defaultdict, deque

def min_transfer_routes(start, breakfast, end, routes):
    # 构建站点到公交路线的映射
    station_to_routes = defaultdict(set)
    
    for i, route in enumerate(routes):
        for station in route:
            station_to_routes[station].add(i)
    
    # 初始化BFS
    queue = deque([(start, -1, False, 0)])  # (当前站点, 当前路线, 是否经过早餐站, 已换乘次数)
    visited = set()  # 记录访问过的站点和路线组合
    visited.add((start, -1, False))
    
    while queue:
        station, route, passed_breakfast, count = queue.popleft()
        
        # 检查是否到达终点并经过了早餐店
        if station == end and passed_breakfast:
            return count
        
        # 遍历所有包含当前站点的公交路线
        for next_route in station_to_routes[station]:
            # 遍历公交路线中的每个站点
            for next_station in routes[next_route]:
                new_passed_breakfast = passed_breakfast or (next_station == breakfast)
                new_state = (next_station, next_route, new_passed_breakfast)
                
                # 检查是否已访问过此状态
                if new_state not in visited:
                    visited.add(new_state)
                    # 如果是同一路线，不增加换乘次数；否则换乘次数+1
                    if next_route == route:
                        queue.append((next_station, next_route, new_passed_breakfast, count))
                    else:
                        queue.append((next_station, next_route, new_passed_breakfast, count + 1))
    
    # 如果没有路径满足条件，返回-1
    return -1

# 示例测试
# start = 1
# breakfast = 9
# end = 15
# routes = [
#     [1, 2, 3, 4, 5],      # 第0条公交路线
#     [3, 6, 7, 8, 9],      # 第1条公交路线
#     [10, 9, 8, 11, 12],   # 第2条公交路线
#     [5, 13, 14, 15],      # 第3条公交路线
#     [4, 12, 14]           # 第4条公交路线
# ]
start = 1
breakfast = 4
end = 5
routes = [
    [1, 2, 3],  # 第0条路线
    [3, 4, 5]   # 第1条路线
]

min_transfer_routes(start, breakfast, end, routes)


2

In [23]:
wall = [[1,2,2,1],[3,1,2],[1,3,2],[2,4],[3,1,2],[1,3,1,1]]
wall1=[[1],[1],[1]]
#输出：2
def help(wall):
    m=len(wall)
    n=sum(wall[0])
    res=float('inf')
    
    wall_new=[]
    for num in wall:
        length=len(num)
        for i in range(1,length):
            num[i]=num[i-1]+num[i]
        wall_new.append(num[:-1])
    print(wall_new)
    for i in range(1,n+1):
        count=0
        for j in range(m):
            if i not in wall_new[j]:
                count+=1
        res=min(res,count)
    return res
help(wall)
            



[[1, 3, 5], [3, 4], [1, 4], [2], [3, 4], [1, 4, 5]]


2

In [16]:
def help(wall):
    m = len(wall)
    n = sum(wall[0])
    res = float('inf')
    wall_new = []
    
    # 构建每行的累积和列表，以标记砖块边缘
    for row in wall:
        cumulative_row = []
        total = 0
        for brick in row[:-1]:  # 不包含最后一个砖块，避免到达墙的末端
            total += brick
            cumulative_row.append(total)
        wall_new.append(cumulative_row)
    
    print("累积墙边缘位置:", wall_new)
    
    # 对于每个可能的边缘位置，计算跨过的行数
    for i in range(1, n):
        count = 0
        for row_edges in wall_new:
            if i not in row_edges:
                count += 1
        res = min(res, count)
    
    return res

# 测试示例
wall = [[1,2,2,1], [3,1,2], [1,3,2], [2,4], [3,1,2], [1,3,1,1]]
print("最小跨越数:", help(wall))


累积墙边缘位置: [[1, 3, 5], [3, 4], [1, 4], [2], [3, 4], [1, 4, 5]]
最小跨越数: 2
